## LCEL = langchain expression languge
Used to chain componaned togather 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x79ad007d6610>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79ad00838c10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi my name is shubham and im Chief AI Engineer")])

AIMessage(content="Hello Shubham, it's nice to meet you!\n\nThat's a very impressive title - Chief AI Engineer. What kind of exciting projects are you working on? \n\nI'm eager to learn more about your work in the field of AI.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 21, 'total_tokens': 79, 'completion_time': 0.105454545, 'prompt_time': 0.002118715, 'queue_time': 0.22979644300000002, 'total_time': 0.10757326}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-37943b6d-ee4a-47cb-92aa-5a0b4aefca38-0', usage_metadata={'input_tokens': 21, 'output_tokens': 58, 'total_tokens': 79})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi my name is shubham and im Chief AI Engineer"),
        AIMessage(content="Hello Shubham, it's a pleasure to meet you! \n\nThat's an impressive title - Chief AI Engineer. What kind of exciting projects are you working on?  \n\nI'm always interested to learn more about how AI is being used in different industries.\n"),
        HumanMessage(content="Hey whats my name and what do i do?")
    ]
)

AIMessage(content="You are Shubham, and you are the Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about your work?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 98, 'total_tokens': 131, 'completion_time': 0.06, 'prompt_time': 0.00425684, 'queue_time': 0.231666459, 'total_time': 0.06425684}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8381475d-ad21-4ca7-b568-561e67af8492-0', usage_metadata={'input_tokens': 98, 'output_tokens': 33, 'total_tokens': 131})

#  Message History

we can use a message history class to wrap our model and make it statful. this will keep track of inputs and outputs of the model,<br>and store them in some datastore future instructions will then load those messages and pass them into the chain as part of the input 

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config = {"configurable":{"session_id":"chat1"}}

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is shubham and im Chief AI Engineer")],config=config
)

In [9]:
response.content

"Hello Shubham, it's nice to meet you!\n\nThat's an impressive title. What kind of exciting AI projects are you working on as Chief AI Engineer?  \n\nI'm always eager to learn more about how AI is being used in different fields.  \n\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="whats my name")],config=config
)

AIMessage(content='Your name is Shubham.  \n\nI remember that from our first interaction. 😊  How can I help you today, Shubham?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 91, 'total_tokens': 123, 'completion_time': 0.058181818, 'prompt_time': 0.00423765, 'queue_time': 0.237280798, 'total_time': 0.062419468}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-87eb5122-0359-4041-88a5-ccd8424a02d5-0', usage_metadata={'input_tokens': 91, 'output_tokens': 32, 'total_tokens': 123})

In [11]:
## change the config chnage the session id

config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What is my name and what do i do?")],config=config1
)
response.content

"As an AI, I have no memory of past conversations and no access to personal information about you. This means I don't know your name or what you do.\n\nCould you tell me your name? I'd love to know!\n"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="my name is shubham and im AI engineer at tesla")],config=config1
)
response.content

"That's awesome, Shubham! It's really cool to meet an AI engineer working at Tesla. What kind of AI projects are you working on there?  \n\nI'm always fascinated to learn more about how AI is being used in real-world applications, especially in such an innovative company.\n"

# prompt templetes 
it help to turn raw user info a format thet the LLM can work with in this case raw users input is just a message which are passing to the LLM <br>lets now makr that a bit more complicated first lets add in system message with some custom instruction , Next well add in more input beside just the message 

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. answer all the questions to the nest of ypur ability "),
        MessagesPlaceholder(variable_name="messages")
    ]
)


chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="hi my name is shubham")]})

AIMessage(content="Hello Shubham!\n\nIt's nice to meet you.  \n\nWhat can I do for you today? I'm ready to answer your questions and help in any way I can. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_time': 0.081818182, 'prompt_time': 0.002355014, 'queue_time': 0.235750175, 'total_time': 0.084173196}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2bde6967-a5ab-4a3b-b494-554d0e5d01e4-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, 'total_tokens': 79})

In [15]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [
        HumanMessage(content="hi my name is shubham")
    ],
    config=config
)

response.content

"Hi Shubham, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away! 😊  What can I do for you today? \n"

In [17]:
## Ading more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are healpfull assistant, answer all questions to the best of your ability in {languge}."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [18]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is shubham")],"languge":"Hindi"})
response.content

'नमस्ते शुभम! \n\nआपका नाम सुनकर बहुत अच्छा लगा। मुझे आपकी मदद करने में खुशी होगी। \n\nआप मुझसे क्या पूछना चाहते हैं? \n'

In [19]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [20]:
config = {"configurable":{"session_id":"chat4"}}
resopnse = with_message_history.invoke(
    {'messages':[HumanMessage(content="hi i am shuhbam")],"languge":"Hindi"},
    config=config
)
response.content

'नमस्ते शुभम! \n\nआपका नाम सुनकर बहुत अच्छा लगा। मुझे आपकी मदद करने में खुशी होगी। \n\nआप मुझसे क्या पूछना चाहते हैं? \n'

In [21]:
resopnse = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name and what do i do ?")],"languge":"Hindi"},
    config=config,
)
resopnse.content

'आपका नाम शुभम है और आप  मुझसे बात कर रहे हैं! \n\nक्या आप मुझे और बताना चाहेंगे कि आप क्या करते हैं? \n\n'

In [22]:
resopnse = with_message_history.invoke(
    {"messages": [HumanMessage(content="my name is shubham and im AI enngineer")],"languge":"Hindi"},
    config=config,
)
resopnse.content

'शुभम,  AI इंजीनियर होना बहुत ही रोमांचक है! \n\nआप किस तरह के AI परियोजनाओं में काम करते हैं? \n'

In [23]:
resopnse = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name and what do i do ?")],"languge":"Hindi"},
    config=config,
)
resopnse.content

'आपका नाम शुभम है और आप एक AI इंजीनियर हैं!  \n\nक्या आपके कोई और सवाल हैं? \n'

## manage conversation history

In [24]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens=70,
    strategy = "last",
    token_counter=model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)
messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="i like vanilla icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/home/shubham/anaconda3/envs/gen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i like vanilla icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)


In [26]:
chain.invoke({
    "messages":messages + [HumanMessage(content="what math problem do i asked for")],
    "languge":"english"
}
)

response.content

'नमस्ते शुभम! \n\nआपका नाम सुनकर बहुत अच्छा लगा। मुझे आपकी मदद करने में खुशी होगी। \n\nआप मुझसे क्या पूछना चाहते हैं? \n'

In [27]:
## lets wrap this in the msg history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
    
)
config={"configurable":{"session_id":"chat5"}}

In [30]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "languge": "English",
    },
    config=config,
)

resopnse.content

'आपका नाम शुभम है और आप एक AI इंजीनियर हैं!  \n\nक्या आपके कोई और सवाल हैं? \n'